In [ ]:
##version1
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    all_read_simhash = []
    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat),dtype=object)  
    for flag,seed in enumerate(hash_seeds):
        kmer_hash_indice = {} 
        for kmer_index in range(kmer_num):
            hash_table[kmer_index,flag]=_hash(kmer_index, seed=seed)

    for read_ind,read_kmer in read_features.items():
        one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
        conc_hash = np.concatenate(one_read_hash)
        simhash = np.where(conc_hash > 0, 1, 0)
        all_read_simhash.append(simhash)
    reads_simhash_array = np.array(all_read_simhash)
  
    return reads_simhash_array 

def hamming_distance(x, y):  
    return np.count_nonzero(x != y)

for repeat in range(50,110,10):
    print(repeat)
    concat_simhash = _get_simhash(read_features,feature_matrix,repeat =repeat, seed = 4829)
    nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
    nbrs.fit(concat_simhash)  
    indices = nbrs.kneighbors(concat_simhash,return_distance=False)
    nbr_indices = indices[:, 1:]

    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)
    dfs.append(df)

new = pd.concat(dfs)
new.to_csv('/home/miaocj/docker_dir/test_simhash.csv',sep='\t')

In [ ]:
##version2 numpy calculate hash values
##需要整合成函数
repeat = 20

hash_seeds = np.array(range(repeat))
kmer_num = feature_matrix.shape[1]
kmer_index = np.array(range(kmer_num))

table = []
for seed in hash_seeds:
    prime1 = 2654435761  # A large prime number
    prime2 = 0x27d4eb2d  # Another large prime, often used in hashing
    hash_value = (kmer_index * prime1) ^ (seed * prime2)
    hash_t = hash_value % (2**32)
    binary_matrix = np.vectorize(np.binary_repr)(hash_t, width=32)
    one_repear_table = np.array([list(row) for row in binary_matrix.flatten()])
    table.append(one_repear_table.astype(int))

table = np.where(table ==0,-1,1)
hash_table1 = np.hstack(table)
print("hash_Table1 es")
hash_table = np.where(hash_table1 ==0,-1,1)
print("hash_table established")

all_read_simhash = []

for read_ind,read_kmer in read_features.items():
    one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
    simhash = np.where(conc_hash > 0, 1, 0)
    all_read_simhash.append(simhash)
reads_simhash_array = np.array(all_read_simhash)


for repeat in range(50,110,10):
    print(repeat)
    concat_simhash = _get_simhash(read_features,feature_matrix,repeat =repeat, seed = 4829)
    nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
    nbrs.fit(concat_simhash)  
    indices = nbrs.kneighbors(concat_simhash,return_distance=False)
    nbr_indices = indices[:, 1:]

    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)
    dfs.append(df)

new = pd.concat(dfs)
new.to_csv('/home/miaocj/docker_dir/test_simhash.csv',sep='\t')